<a href="https://colab.research.google.com/github/Msingi-AI/Sauti-Ya-Kenya/blob/main/colab/train_tts_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sauti Ya Kenya - TTS Training (GPU Version)

This notebook trains the Swahili TTS model using GPU acceleration.

## Setup Steps:
1. Mount Google Drive
2. Install dependencies
3. Upload dataset files
4. Train and evaluate model

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
if not torch.cuda.is_available():
    raise RuntimeError("No GPU found! Go to Runtime > Change runtime type and select GPU")

print(f"Using GPU: {torch.cuda.get_device_name(0)}")
DEVICE = torch.device('cuda')

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/Msingi-AI/Sauti-Ya-Kenya.git
%cd Sauti-Ya-Kenya
!pip install -r requirements.txt

In [ ]:
# Prepare dataset directory
import os
dataset_dir = "cv-corpus-21.0-delta-2025-03-14/sw"
clips_dir = os.path.join(dataset_dir, "clips")
os.makedirs(clips_dir, exist_ok=True)

# Upload dataset files
from google.colab import files
print("Please upload the following files in order:")
print("1. validated_sentences.tsv")
print("2. clip_durations.tsv")
print("3. All clips_part_*.zip files")

# Handle TSV files first
print("\nUploading TSV files...")
uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.tsv'):
        dest_path = os.path.join(dataset_dir, filename)
        with open(dest_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Saved {filename}")

# Now handle zip files
print("\nUploading and extracting clip files...")
while True:
    try:
        uploaded = files.upload()
        if not uploaded:
            break
            
        for filename in uploaded.keys():
            if filename.startswith('clips_part_') and filename.endswith('.zip'):
                print(f"Extracting {filename}...")
                import zipfile
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(dataset_dir)
                os.remove(filename)  # Clean up zip file
                print(f"Extracted {filename}")
    except Exception as e:
        if "No file selected" in str(e):
            break
        else:
            raise e

print("\nDataset preparation complete!")
print(f"Number of audio files: {len(os.listdir(clips_dir))}")

In [ ]:
# Process dataset with GPU acceleration
!CUDA_VISIBLE_DEVICES=0 python src/prepare_local_dataset.py \
    --dataset_path "cv-corpus-21.0-delta-2025-03-14/sw" \
    --output_dir "processed_data" \
    --clips_path "clips" \
    --sentences_file "validated_sentences.tsv" \
    --durations_file "clip_durations.tsv" \
    --use_gpu

In [ ]:
# Run data augmentation with GPU
!CUDA_VISIBLE_DEVICES=0 python src/augment_data.py \
    --data_dir "processed_data" \
    --output_dir "augmented_data" \
    --num_augmentations 3 \
    --use_gpu

In [ ]:
# Set up GPU-optimized training configuration
import json

config = {
    "train_data_dir": "augmented_data",
    "batch_size": 32,           # Optimized for GPU
    "learning_rate": 0.001,
    "num_epochs": 100,
    "save_every": 10,
    "device": "cuda",
    "max_len": 10000,          # Based on our Swahili optimization
    "checkpoint_dir": "checkpoints",
    "gradient_accumulation_steps": 2,  # Help with GPU memory
    "mixed_precision": True,    # Use FP16 for faster training
    "num_workers": 4,          # Parallel data loading
    "pin_memory": True         # Faster GPU data transfer
}

with open('config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("Training configuration:")
for k, v in config.items():
    print(f"{k}: {v}")

In [ ]:
# Start GPU training
!CUDA_VISIBLE_DEVICES=0 python src/train.py --config config.json

In [ ]:
# Run evaluation
!CUDA_VISIBLE_DEVICES=0 python src/evaluation.py \
    --model_path "checkpoints/best.pt" \
    --test_data "augmented_data/test" \
    --output_dir "evaluation_results" \
    --use_gpu

In [ ]:
# Save model to Google Drive
drive_path = "/content/drive/MyDrive/Sauti-Ya-Kenya/models"
os.makedirs(drive_path, exist_ok=True)

# Copy best model and results
import shutil
shutil.copy("checkpoints/best.pt", f"{drive_path}/best.pt")
shutil.copytree("evaluation_results", f"{drive_path}/evaluation_results", dirs_exist_ok=True)

print("Model and evaluation results saved to Google Drive")